# FINAL MOVE

In [ ]:
import os
# import nibabel as nib
# from nibabel.testing import data_path
import numpy as np
import cv2 
from matplotlib import pyplot as plt 
import sys
import shutil


In [ ]:
import pandas as pd

def createMetaCombinedString(meta_df):
    meta_image = meta_df.loc[:,"Image Data ID"]
    meta_subject = meta_df.loc[:,"Subject"]
    meta_visit = meta_df.loc[:,"Visit"]

    meta_combined= []
    i=0
    for i in range(len(meta_image)):
        # combined = str(meta_subject[i])+"-"+str(meta_series[i])+"-"+str(meta_image[i])
        combined = str(meta_subject[i])+"-"+str(meta_image[i]) #to check data availability based on subject and image id
        meta_combined.append(combined)
        i+=1

    return meta_combined

def exportCSV(meta_dict, title):
    temp_meta_df = pd.DataFrame(meta_dict) #convert dict to dataframe
    meta_csv_file = "./TempMeta/{}.csv".format(title)
    with open(meta_csv_file, mode='w') as f:
        temp_meta_df.to_csv(f)

    return temp_meta_df

In [ ]:

from pathlib import Path
# path = "C:/Users/Brain-LPDP/Documents/DEWINDA PMDSU/Publication/2023 - Research Visit KHU/Experiment/ADNI1 All Data/ADNI"
# path ="./ADNI/"

def movePreprocessed(meta_df, path, seq, cond, tesla=3, divider="raw_"):
    # target_meta = "./metadata/"
    target_path = "./preprocessed/"
    # path = "./preprocessed_old/{}/{}/".format(seq, cond)

    path = path + "/{}/{}/".format(seq, cond)
    print(path)
    meta_nums = []
    result = list(Path(path).glob('**/wm*.nii'))
    unique = set(result)
    print("---------\n{}w-{}\nOriginal number of files: {}\nUnique result:{}".format(seq, cond, len(result), len(unique)))

    sim = 0
    # ctr = 0
    notsim = 0
    j=0
    target_dir = os.path.join(target_path, seq)
    target_dir = os.path.join(target_dir, cond)

    meta_combined = createMetaCombinedString(meta_df)
    meta_dict = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    meta_dict.clear()

    meta_dict = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    
    notsim = 0
    for j in range(len(meta_combined)):

        flag = 0 #flag that there is matched data
        for f in result[:]:
            pathToFile = str(f)
            # print(pathToFile)
            fileName = pathToFile.split("\\")[5] #use "\\" if "/" does not work
            # print(fileName)
            fileNameNoExt = fileName.split(".nii")[0]

            part = fileNameNoExt.split('_MR')
            id_subject = part[0].split("ADNI_")[1]            
            part = part[1].split(divider)[1]
            id_series = part.split("_S")[1]
            id_series = id_series.split("_I")[0]
            id_image = fileNameNoExt.split("_I")[1]
        
            combinedIDs = id_subject+"-I"+id_image
            subdirName = id_subject+"-"+id_series+"-"+id_image
            
            
            if combinedIDs in meta_combined[j]: #Files already preprocessed             
                isExist = os.path.exists(target_dir)    

                if isExist == False:
                    os.mkdir(target_dir)
                
                shutil.copy(f,target_dir+"/{}-".format(j)+fileName)
                
                flag = 1

        if flag==1: #Files are preprocessed
            sim+=1      

        else: #Listing the not-yet-preprocessed files
            meta_dict["Image Data ID"].append(meta_df.loc[j,"Image Data ID"])
            meta_dict["Subject"].append(meta_df.loc[j,"Subject"])
            meta_dict["Group"].append(meta_df.loc[j,"Group"])
            meta_dict["Sex"].append(meta_df.loc[j,"Sex"])
            meta_dict["Age"].append(meta_df.loc[j,"Age"])
            meta_dict["Visit"].append(meta_df.loc[j,"Visit"])
            meta_dict["Modality"].append(meta_df.loc[j,"Modality"])
            meta_dict["Description"].append(meta_df.loc[j,"Description"])
            meta_dict["Type"].append(meta_df.loc[j,"Type"])
            meta_dict["Acq Date"].append(meta_df.loc[j,"Acq Date"])
            meta_dict["Format"].append( meta_df.loc[j,"Format"])   
            meta_nums.append(j)
            notsim +=1          
                
        j+=1
    # ctr+=1   
    # print(ctr)
    print("Total {}w-{} data is {} and not sim is {}".format(seq, cond, sim, notsim))
    return meta_dict, meta_nums

def move2preprocess(meta_df, seq, cond, tesla=3, divider="raw_"):
    nii_path = "./{}T/{}/{}/".format(tesla, seq, cond)

    target_meta = "./TempMeta/"
    target_path = "./TempData/"

    result = sorted(list(Path(nii_path).glob('**/wm*.nii')))
    unique = set(result)
    print("---------\n{}w-{}\nOriginal number: {}\nUnique result:{}".format(seq, cond, len(result), len(unique)))

    sim = 0
    ctr = 0
    meta_combined = createMetaCombinedString(meta_df)

    for f in result[:]:
        pathToFile = str(f)
        fileName = pathToFile.split("/")[7] #use "\\" if "/" does not work 7 if from folder 3T/seq/group
        fileNameNoExt = fileName.split(".nii")[0]
        
        part = fileNameNoExt.split('_MR')
        id_subject = part[0].split("ADNI_")[1]
        part = part[1].split(divider)[1]
        id_series = part.split("_S")[1]
        id_series = id_series.split("_I")[0]
        id_image = fileNameNoExt.split("_I")[1]
    
        combinedIDs = id_subject+"-I"+id_image
        subdirName = id_subject+"-"+id_series+"-"+id_image
        
        target_dir = os.path.join(target_path, seq)
        target_dir = os.path.join(target_dir, cond)
        target_dir = os.path.join(target_dir, subdirName)
        isExist = os.path.exists(target_dir)    

        j = 0
        for j in range(len(meta_combined)):
            if meta_combined[j] in combinedIDs:

                if isExist == False:
                    os.mkdir(target_dir)
                # print(target_dir)
                # print("File {} copied to target folder".format(fileName))
                shutil.copy(f,target_dir+"/"+fileName)

                sim+=1
            j+=1
        ctr+=1    
        # print("File {}".format(ctr))
    print("Total {}w - {} data is {}".format(seq, cond, sim))

def freemove(path, seq, cond, tesla=3): #move anything based on filename
    # target_meta = "./metadata/"
    target_path = "./arnold/"
    # path = "./preprocessed_old/{}/{}/".format(seq, cond)
    target_dir = os.path.join(target_path, seq)
    target_dir = os.path.join(target_dir, cond)

    path = path + "/{}/{}/".format(seq, cond)
   
    meta_nums = []
    result = sorted(list(Path(path).glob('**/*.nii'))) #modify it to catch certain chars
    unique = set(result)
    print("---------\n{}w-{}\nOriginal number of files: {}\nUnique result:{}".format(seq, cond, len(result), len(unique)))
    
    j =0
    for f in result[:]:
        pathToFile = str(f)
        # print(pathToFile)
        fileName = pathToFile.split("\\")[4] #use "\\" if "/" does not work
        # print(fileName)
        
        # if "wm" not in fileName and "y_" not in fileName and "mwp" not in fileName and "p0" not in fileName: #Files already preprocessed             
        if "ADNI" in fileName:
            print(fileName)
            isExist = os.path.exists(target_dir)    

            if isExist == False:
                os.mkdir(target_dir)
            
            shutil.copy(f,target_dir+"/{}-".format(j)+fileName)
            j=j+1

In [ ]:
freemove("./preprocessed/", "T2", "AD")
freemove("./preprocessed", "T2", "MCI")
freemove("./preprocessed/", "T2", "CN")

# Moving Already Preprocessed Files
------
Moving already prerpocessed files from the source (previous preprocessed folder) to the target folder and keep the list of the image that is not yet preprocessed

**Current Steps:**
1. Preprocessed files are taken from the folder `/preprocess_old` and saved to `/preprocessed` 
<br><br>-The preprocessed files are saved with this filename: `/preprocessed/{meta_id}_{filename}.nii.gz`
<br>-`{meta_id}` is the serial number of the file shown in the csv list
<br>-The idea of giving `{meta_id}` is to handle pairing of T1 and T2 data together


2. The list of not yet preprocessed files is saved as `TempMeta/To-Be-Preprocessed_{seq}w_{group}.csv`
-----
**Further steps:**
The list in Step #2 will be used to move the to-be-prerprocessed files to folder "/TempData" <br> This way the files can be transferred to the windows computer to be preprocessed

### Moving Old Preprocessed Files

In [ ]:
# tesla = 3 #1.5 or 3
# downloadDate = "12_05_2022"
# path = "./{}T/{}/{}/".format(tesla, seq, cond)

# path = "./preprocessed_old/{}/{}/".format(seq, cond)

seq = "T1"
cond= "AD" #condition is AD or CN or MCI
meta_path_1_ad = "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
cond= "MCI" #condition is AD or CN or MCI
meta_path_1_mci = "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
cond= "CN" #condition is AD or CN or MCI
meta_path_1_cn =  "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
seq = "T2"
cond= "AD" #condition is AD or CN or MCI
meta_path_2_ad = "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
cond= "MCI" #condition is AD or CN or MCI
meta_path_2_mci =  "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
cond= "CN" #condition is AD or CN or MCI
meta_path_2_cn = "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
# path = "./data/{}/{}/".format(seq, cond)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
print(datetime.now().strftime('%Y%m%d-%H%M%S'))
# meta_path = "./Metadata/metadata_final.csv"
meta_df_1_ad = pd.read_csv(meta_path_1_ad, delimiter=',')
meta_df_1_mci = pd.read_csv(meta_path_1_mci, delimiter=',')
meta_df_1_cn = pd.read_csv(meta_path_1_cn, delimiter=',')
meta_df_2_ad = pd.read_csv(meta_path_2_ad, delimiter=',')
meta_df_2_mci = pd.read_csv(meta_path_2_mci, delimiter=',')
meta_df_2_cn = pd.read_csv(meta_path_2_cn, delimiter=',')

In [ ]:
target_path = "./preprocessed/"
path = "./preprocessed_old"

dict_1_ad, meta_num_1_ad = movePreprocessed(meta_df_1_ad, path, "T1", "AD")
dict_1_mci, meta_num_1_mci = movePreprocessed(meta_df_1_mci, path, "T1", "MCI")
dict_1_cn, meta_num_1_cn = movePreprocessed(meta_df_1_cn, path, "T1", "CN")
dict_2_ad, meta_num_2_ad = movePreprocessed(meta_df_2_ad, path,  "T2", "AD")
dict_2_mci, meta_num_2_mci = movePreprocessed(meta_df_2_mci, path, "T2", "MCI")
dict_2_cn, meta_num_2_cn = movePreprocessed(meta_df_2_cn, path,  "T2", "CN")

In [ ]:
print(meta_num_1_ad)
print(meta_num_1_mci)
print(meta_num_1_cn)
print(meta_num_2_ad)
print(meta_num_2_mci)
print(meta_num_2_cn)

In [ ]:
tobe_df_1_ad = exportCSV(dict_1_ad, title="To-Be-Preprocessed_{}w_{}".format("T1", "AD"))
tobe_df_1_mci = exportCSV(dict_1_mci, title="To-Be-Preprocessed_{}w_{}".format("T1", "MCI"))
tobe_df_1_cn = exportCSV(dict_1_cn, title="To-Be-Preprocessed_{}w_{}".format("T1", "CN"))
tobe_df_2_ad = exportCSV(dict_2_ad, title="To-Be-Preprocessed_{}w_{}".format("T2", "AD"))
tobe_df_2_mci = exportCSV(dict_2_mci, title="To-Be-Preprocessed_{}w_{}".format("T2", "MCI"))
tobe_df_2_cn = exportCSV(dict_2_cn, title="To-Be-Preprocessed_{}w_{}".format("T2", "CN"))


### Moving Additional Preprocessed Files

In [ ]:
# tesla = 3 #1.5 or 3
# downloadDate = "12_05_2022"
# path = "./{}T/{}/{}/".format(tesla, seq, cond)

path = "./preprocessed_addition/{}/{}/".format(seq, cond)

seq = "T1"
cond= "AD" #condition is AD or CN or MCI
meta_path_1_ad = "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
cond= "MCI" #condition is AD or CN or MCI
meta_path_1_mci = "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
cond= "CN" #condition is AD or CN or MCI
meta_path_1_cn =  "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
seq = "T2"
cond= "AD" #condition is AD or CN or MCI
meta_path_2_ad = "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
cond= "MCI" #condition is AD or CN or MCI
meta_path_2_mci =  "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
cond= "CN" #condition is AD or CN or MCI
meta_path_2_cn = "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
# path = "./data/{}/{}/".format(seq, cond)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
print(datetime.now().strftime('%Y%m%d-%H%M%S'))
# meta_path = "./Metadata/metadata_final.csv"
meta_df_1_ad = pd.read_csv(meta_path_1_ad, delimiter=',')
meta_df_1_mci = pd.read_csv(meta_path_1_mci, delimiter=',')
meta_df_1_cn = pd.read_csv(meta_path_1_cn, delimiter=',')
meta_df_2_ad = pd.read_csv(meta_path_2_ad, delimiter=',')
meta_df_2_mci = pd.read_csv(meta_path_2_mci, delimiter=',')
meta_df_2_cn = pd.read_csv(meta_path_2_cn, delimiter=',')

In [ ]:
target_path = "./preprocessed/"
path = "./TempData"

# dict_1_ad, meta_num_1_ad = movePreprocessed(meta_df_1_ad, path, "T1", "AD")
dict_1_mci, meta_num_1_mci = movePreprocessed(meta_df_1_mci, path, "T1", "MCI")
dict_1_cn, meta_num_1_cn = movePreprocessed(meta_df_1_cn, path, "T1", "CN")
# dict_2_ad, meta_num_2_ad = movePreprocessed(meta_df_2_ad, path,  "T2", "AD")
# dict_2_mci, meta_num_2_mci = movePreprocessed(meta_df_2_mci, path, "T2", "MCI")
# dict_2_cn, meta_num_2_cn = movePreprocessed(meta_df_2_cn, path,  "T2", "CN")

# Moving Files to be Preprocessed

----
Moving files that are not yet preprocessed to the target folder, based on the list stored in metadata generated from the previous step. 

**Current Steps:**
1. The not yet preprocessed files re taken from the original downloaded folder `/3T/` and saved to `/TempData/` 
<br><br>-The files are saved with this filename: `/TempMeta/{filename}.nii.gz`

2. The data will be preprocessed in windows PC using MATLAB, therefore need to be put inside a decent sub-directory that contains `{subject id}-{series id}-{image id}`
-----
**Further steps:**
 After the data get preprocessed, we can go on with the third section that can be performed using the functions and algorithms in the first section

In [ ]:
tesla = 3 #1.5 or 3
# downloadDate = "12_05_2022"
path = "./{}T/{}/{}/".format(tesla, seq, cond)
# path = "./preprocessed_old/{}/{}/".format(seq, cond)

seq = "T1"
cond= "AD" #condition is AD or CN or MCI
meta_path_1_ad = "./TempMeta/To-Be-Preprocessed_{}w_{}.csv".format(seq, cond)
cond= "MCI" #condition is AD or CN or MCI
meta_path_1_mci = "./TempMeta/To-Be-Preprocessed_{}w_{}.csv".format(seq, cond)
cond= "CN" #condition is AD or CN or MCI
meta_path_1_cn = "./TempMeta/To-Be-Preprocessed_{}w_{}.csv".format(seq, cond)
seq = "T2"
cond= "AD" #condition is AD or CN or MCI
meta_path_2_ad = "./TempMeta/To-Be-Preprocessed_{}w_{}.csv".format(seq, cond)
cond= "MCI" #condition is AD or CN or MCI
meta_path_2_mci = "./TempMeta/To-Be-Preprocessed_{}w_{}.csv".format(seq, cond)
cond= "CN" #condition is AD or CN or MCI
meta_path_2_cn = "./TempMeta/To-Be-Preprocessed_{}w_{}.csv".format(seq, cond)
# path = "./data/{}/{}/".format(seq, cond)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
print(datetime.now().strftime('%Y%m%d-%H%M%S'))
# meta_path = "./Metadata/metadata_final.csv"
meta_df_1_ad = pd.read_csv(meta_path_1_ad, delimiter=',')
meta_df_1_mci = pd.read_csv(meta_path_1_mci, delimiter=',')
meta_df_1_cn = pd.read_csv(meta_path_1_cn, delimiter=',')
meta_df_2_ad = pd.read_csv(meta_path_2_ad, delimiter=',')
meta_df_2_mci = pd.read_csv(meta_path_2_mci, delimiter=',')
meta_df_2_cn = pd.read_csv(meta_path_2_cn, delimiter=',')

In [ ]:
# move2preprocess(meta_df_1_ad, "T1", "AD")
move2preprocess(meta_df_1_mci, "T1", "MCI")
move2preprocess(meta_df_1_cn, "T1", "CN")
# move2preprocess(meta_df_2_ad, "T2", "AD")
# move2preprocess(meta_df_2_mci, "T2", "MCI")
# move2preprocess(meta_df_2_cn, "T2", "CN")

# Data Separation For Robustness Evaluation

The idea is to move the new datasets (hold-out data) for robustness evaluation to be preprocessed.
The hold-out datasets include 3T and 1.5T MRI. Hold-out data has never been used as the training data (the initial 5-fold data) --refer to `Balanced_Meta_{seq}w_{group}` for comparison.

In [ ]:

from pathlib import Path
# path = "C:/Users/Brain-LPDP/Documents/DEWINDA PMDSU/Publication/2023 - Research Visit KHU/Experiment/ADNI1 All Data/ADNI"
# path ="./ADNI/"

def find2separate(meta_df, ref_df, seq, tesla=3):
    nii_path = "./DataOri/{}T/{}/".format(tesla, seq)

    result = sorted(list(Path(nii_path).glob('**/*.nii')))
    unique = set(result)
    print("---------\n{}w\nOriginal number: {}\nUnique result:{}".format(seq, len(result), len(unique)))

    sim = 0
    notsim = 0
    ctr = 0
    meta_combined = createMetaCombinedString(meta_df)
    ref_combined = createMetaCombinedString(ref_df)

    meta_dict_ad = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    meta_dict_ad.clear()
    meta_dict_mci = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    meta_dict_mci.clear()
    meta_dict_cn = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    meta_dict_cn.clear()
    
    meta_dict_ad = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    meta_dict_mci = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    meta_dict_cn = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    
    #Prepared array to be exported to CSV (separated per class)
    meta_nums_ad = []
    meta_nums_mci = []
    meta_nums_cn = []

    #Loop to check if there is already used data
    for j in range(len(meta_combined)):
        flag = 0 #flag that there is matched data

        for k in range(len(ref_combined)):

            # if ref_combined[k] in meta_combined[j]: #Similarities based on subject and imageid
            if meta_df.loc[j,"Subject"] in ref_df.loc[k,"Subject"]:  #similarities based on subject id only
                print("{} and {}".format(meta_combined[j], ref_combined[k]))
                flag = 1

        if flag==1: #Files are already used
            print("flag")
            sim = sim + 1    

        else: #Listing the not-yet-used files to be preprocessed
            if "AD" in meta_df.loc[j,"Group"]:
                meta_dict_ad["Image Data ID"].append(meta_df.loc[j,"Image Data ID"])
                meta_dict_ad["Subject"].append(meta_df.loc[j,"Subject"])
                meta_dict_ad["Group"].append(meta_df.loc[j,"Group"])
                meta_dict_ad["Sex"].append(meta_df.loc[j,"Sex"])
                meta_dict_ad["Age"].append(meta_df.loc[j,"Age"])
                meta_dict_ad["Visit"].append(meta_df.loc[j,"Visit"])
                meta_dict_ad["Modality"].append(meta_df.loc[j,"Modality"])
                meta_dict_ad["Description"].append(meta_df.loc[j,"Description"])
                meta_dict_ad["Type"].append(meta_df.loc[j,"Type"])
                meta_dict_ad["Acq Date"].append(meta_df.loc[j,"Acq Date"])
                meta_dict_ad["Format"].append( meta_df.loc[j,"Format"])

            elif "MCI" in meta_df.loc[j,"Group"]:
                meta_dict_mci["Image Data ID"].append(meta_df.loc[j,"Image Data ID"])
                meta_dict_mci["Subject"].append(meta_df.loc[j,"Subject"])
                meta_dict_mci["Group"].append(meta_df.loc[j,"Group"])
                meta_dict_mci["Sex"].append(meta_df.loc[j,"Sex"])
                meta_dict_mci["Age"].append(meta_df.loc[j,"Age"])
                meta_dict_mci["Visit"].append(meta_df.loc[j,"Visit"])
                meta_dict_mci["Modality"].append(meta_df.loc[j,"Modality"])
                meta_dict_mci["Description"].append(meta_df.loc[j,"Description"])
                meta_dict_mci["Type"].append(meta_df.loc[j,"Type"])
                meta_dict_mci["Acq Date"].append(meta_df.loc[j,"Acq Date"])
                meta_dict_mci["Format"].append( meta_df.loc[j,"Format"])
            else:
                meta_dict_cn["Image Data ID"].append(meta_df.loc[j,"Image Data ID"])
                meta_dict_cn["Subject"].append(meta_df.loc[j,"Subject"])
                meta_dict_cn["Group"].append(meta_df.loc[j,"Group"])
                meta_dict_cn["Sex"].append(meta_df.loc[j,"Sex"])
                meta_dict_cn["Age"].append(meta_df.loc[j,"Age"])
                meta_dict_cn["Visit"].append(meta_df.loc[j,"Visit"])
                meta_dict_cn["Modality"].append(meta_df.loc[j,"Modality"])
                meta_dict_cn["Description"].append(meta_df.loc[j,"Description"])
                meta_dict_cn["Type"].append(meta_df.loc[j,"Type"])
                meta_dict_cn["Acq Date"].append(meta_df.loc[j,"Acq Date"])
                meta_dict_cn["Format"].append( meta_df.loc[j,"Format"]) 

            notsim +=1          
        j+=1
        
    print("Total of used {}w data is {}".format(seq, sim))
    return meta_dict_ad, meta_dict_mci, meta_dict_cn

def move2separate(meta_df, seq, tesla=3, ONLY_BASELINE=False, divider="Br_"):
    nii_path = "./DataOri/{}T/{}/".format(tesla, seq)
    target_path = "./DataSep/"

    tesla = str(tesla) + "T"
    result = sorted(list(Path(nii_path).glob('**/*.nii')))
    unique = set(result)
    print("---------\n{}w\nOriginal number: {}\nUnique result:{}".format(seq, len(result), len(unique)))

    sim = 0
    ctr = 0
    meta_combined = createMetaCombinedString(meta_df)

    #Initialize dict to save metadata
    meta_dict_ad = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    meta_dict_mci = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    meta_dict_cn = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}

    for f in result[:]:
        pathToFile = str(f)
        #print(pathToFile)
        fileName = pathToFile.split("\\")[7] #use "\\" if "/" does not work 7 if from folder 3T/seq/group
        fileNameNoExt = fileName.split(".nii")[0]
        #print(fileNameNoExt)
        part = fileNameNoExt.split('_MR')
        id_subject = part[0].split("ADNI_")[1]
        part = part[1].split(divider)[1] #raw if T2, Br if T1?
        id_series = part.split("_S")[1]
        id_series = id_series.split("_I")[0]
        id_image = fileNameNoExt.split("_I")[1]
    
        combinedIDs = id_subject+"-I"+id_image
        subdirName = id_subject+"-"+id_series+"-"+id_image
        
        j = 0
        for j in range(len(meta_combined)):
            flag = 0
            if meta_combined[j] in combinedIDs:
                target_dir = os.path.join(target_path, tesla)
                target_dir = os.path.join(target_dir, meta_df.loc[j,"Group"])
                target_dir = os.path.join(target_dir, subdirName)
                
                isExist = os.path.exists(target_dir)
                
                if ONLY_BASELINE: #adjust the conditions
                    # if "AD" in meta_df.loc[j, "Group"]:
                    if "sc" in meta_df.loc[j, "Visit"] and "MPR-R" not in meta_df.loc[j, "Description"]: #Take only screening for 15T MRI
                    # if "bl" in meta_df.loc[j, "Visit"] or "m06" in meta_df.loc[j, "Visit"] or "m12" in meta_df.loc[j, "Visit"]: #Take only baseline from MCI and CN
                        if isExist == False:
                            os.mkdir(target_dir)
                        # print(target_dir)
                        print("File {} copied to target folder".format(fileName))
                        shutil.copy(f,target_dir+"/"+fileName)
                        flag=1
                else:
                    if isExist == False:
                        os.mkdir(target_dir)
                    # print(target_dir)
                    print("File {} copied to target folder".format(fileName))
                    shutil.copy(f,target_dir+"/"+fileName)
                    flag=1    
                sim+=1
            
            if flag==1: #Files are already used
                if "AD" in meta_df.loc[j,"Group"]:
                    meta_dict_ad["Image Data ID"].append(meta_df.loc[j,"Image Data ID"])
                    meta_dict_ad["Subject"].append(meta_df.loc[j,"Subject"])
                    meta_dict_ad["Group"].append(meta_df.loc[j,"Group"])
                    meta_dict_ad["Sex"].append(meta_df.loc[j,"Sex"])
                    meta_dict_ad["Age"].append(meta_df.loc[j,"Age"])
                    meta_dict_ad["Visit"].append(meta_df.loc[j,"Visit"])
                    meta_dict_ad["Modality"].append(meta_df.loc[j,"Modality"])
                    meta_dict_ad["Description"].append(meta_df.loc[j,"Description"])
                    meta_dict_ad["Type"].append(meta_df.loc[j,"Type"])
                    meta_dict_ad["Acq Date"].append(meta_df.loc[j,"Acq Date"])
                    meta_dict_ad["Format"].append( meta_df.loc[j,"Format"])   
                    
                elif "MCI" in meta_df.loc[j,"Group"]:
                    meta_dict_mci["Image Data ID"].append(meta_df.loc[j,"Image Data ID"])
                    meta_dict_mci["Subject"].append(meta_df.loc[j,"Subject"])
                    meta_dict_mci["Group"].append(meta_df.loc[j,"Group"])
                    meta_dict_mci["Sex"].append(meta_df.loc[j,"Sex"])
                    meta_dict_mci["Age"].append(meta_df.loc[j,"Age"])
                    meta_dict_mci["Visit"].append(meta_df.loc[j,"Visit"])
                    meta_dict_mci["Modality"].append(meta_df.loc[j,"Modality"])
                    meta_dict_mci["Description"].append(meta_df.loc[j,"Description"])
                    meta_dict_mci["Type"].append(meta_df.loc[j,"Type"])
                    meta_dict_mci["Acq Date"].append(meta_df.loc[j,"Acq Date"])
                    meta_dict_mci["Format"].append( meta_df.loc[j,"Format"])   
                    
                else:
                    meta_dict_cn["Image Data ID"].append(meta_df.loc[j,"Image Data ID"])
                    meta_dict_cn["Subject"].append(meta_df.loc[j,"Subject"])
                    meta_dict_cn["Group"].append(meta_df.loc[j,"Group"])
                    meta_dict_cn["Sex"].append(meta_df.loc[j,"Sex"])
                    meta_dict_cn["Age"].append(meta_df.loc[j,"Age"])
                    meta_dict_cn["Visit"].append(meta_df.loc[j,"Visit"])
                    meta_dict_cn["Modality"].append(meta_df.loc[j,"Modality"])
                    meta_dict_cn["Description"].append(meta_df.loc[j,"Description"])
                    meta_dict_cn["Type"].append(meta_df.loc[j,"Type"])
                    meta_dict_cn["Acq Date"].append(meta_df.loc[j,"Acq Date"])
                    meta_dict_cn["Format"].append( meta_df.loc[j,"Format"])   
                
            j+=1
            
    print("Total {}w data moved is {}".format(seq, sim))
    return meta_dict_ad, meta_dict_mci, meta_dict_cn

### Check Already Used Data

In [ ]:
Tesla = 15

# meta_path = "./Metadata/ADNI1_Baseline_3T_6_08_2023.csv" #Meta path for the target data
# meta_path = "./Metadata/ADNI1_Complete_3Yr_3T_6_08_2023.csv"
meta_path = "./Metadata/ADNI1_Complete_1Yr_15T_6_08_2023.csv"

# Reference metadata (Already used for training)
meta_ad = "./Metadata/Balanced_Meta_T1w_AD.csv"
meta_mci = "./Metadata/Balanced_Meta_T1w_MCI.csv"
meta_cn = "./Metadata/Balanced_Meta_T1w_CN.csv"

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
print(datetime.now().strftime('%Y%m%d-%H%M%S'))
# meta_path = "./Metadata/metadata_final.csv"
meta_df = pd.read_csv(meta_path, delimiter=',')
meta_df_ad = pd.read_csv(meta_ad, delimiter=',')
meta_df_mci = pd.read_csv(meta_mci, delimiter=',')
meta_df_cn = pd.read_csv(meta_cn, delimiter=',')

In [ ]:
combined_df = pd.concat([meta_df_ad, meta_df_mci, meta_df_cn], ignore_index=True)

In [ ]:
dict_1_ad, dict_1_mci, dict_1_cn = find2separate(meta_df,combined_df, "T1", Tesla)

In [ ]:
tobe_df_1_ad = exportCSV(dict_1_ad, title="HoldOut_{}w_{}_{}T".format("T1", "AD", Tesla))
tobe_df_1_mci = exportCSV(dict_1_mci, title="HoldOut_{}w_{}_{}T".format("T1", "MCI", Tesla))
tobe_df_1_cn = exportCSV(dict_1_cn, title="HoldOut_{}w_{}_{}T".format("T1", "CN", Tesla))

### Move New Data To Be Preprocessed

In [ ]:
Tesla = 15

# meta_path = "./Metadata/ADNI1_Baseline_3T_6_08_2023.csv" #Meta path for the target data
# meta_path = "./Metadata/ADNI1_Complete_3Yr_3T_6_08_2023.csv"
meta_path = "./Metadata/ADNI1_Complete_1Yr_15T_6_08_2023.csv"

# Reference metadata (Already used for training)
meta_ad = "./TempMeta/HoldOut_T1w_AD_{}T.csv".format(Tesla)
meta_mci = "./TempMeta/HoldOut_T1w_MCI_{}T.csv".format(Tesla)
meta_cn = "./TempMeta/HoldOut_T1w_CN_{}T.csv".format(Tesla)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
print(datetime.now().strftime('%Y%m%d-%H%M%S'))
# meta_path = "./Metadata/metadata_final.csv"
meta_df = pd.read_csv(meta_path, delimiter=',')
meta_df_ad = pd.read_csv(meta_ad, delimiter=',')
meta_df_mci = pd.read_csv(meta_mci, delimiter=',')
meta_df_cn = pd.read_csv(meta_cn, delimiter=',')

In [ ]:
combined_df = pd.concat([meta_df_ad, meta_df_mci, meta_df_cn], ignore_index=True)

In [ ]:
dict_1_ad, dict_1_mci, dict_1_cn = move2separate(combined_df, "T1", Tesla, ONLY_BASELINE=True)

In [ ]:
tobe_df_1_ad = exportCSV(dict_1_ad, title="HoldOut_Cleaned_{}w_{}_{}T".format("T1", "AD", Tesla))
tobe_df_1_mci = exportCSV(dict_1_mci, title="HoldOut_Cleaned_{}w_{}_{}T".format("T1", "MCI", Tesla))
tobe_df_1_cn = exportCSV(dict_1_cn, title="HoldOut_Cleaned_{}w_{}_{}T".format("T1", "CN", Tesla))

### Move Preprocessed Hold-out Data

In [ ]:
# tesla = 3 #1.5 or 3
# downloadDate = "12_05_2022"
# path = "./{}T/{}/{}/".format(tesla, seq, cond)

# path = "./preprocessed_old/{}/{}/".format(seq, cond)

seq = "T1"
Tesla = 15
cond= "AD" #condition is AD or CN or MCI
meta_path_1_ad = "./TempMeta/HoldOut_Balanced_{}w_{}_{}T.csv".format(seq, cond, Tesla)
cond= "MCI" #condition is AD or CN or MCI
meta_path_1_mci = "./TempMeta/HoldOut_Balanced_{}w_{}_{}T.csv".format(seq, cond, Tesla)
cond= "CN" #condition is AD or CN or MCI
meta_path_1_cn = "./TempMeta/HoldOut_Balanced_{}w_{}_{}T.csv".format(seq, cond, Tesla)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
print(datetime.now().strftime('%Y%m%d-%H%M%S'))
# meta_path = "./Metadata/metadata_final.csv"
meta_df_1_ad = pd.read_csv(meta_path_1_ad, delimiter=',')
meta_df_1_mci = pd.read_csv(meta_path_1_mci, delimiter=',')
meta_df_1_cn = pd.read_csv(meta_path_1_cn, delimiter=',')

In [ ]:
target_path = "./preprocessed/"
path = "./TempData"

dict_1_ad, meta_num_1_ad = movePreprocessed(meta_df_1_ad, path, "15T", "AD", divider="Br_")
dict_1_mci, meta_num_1_mci = movePreprocessed(meta_df_1_mci, path, "15T", "MCI", divider="Br_")
dict_1_cn, meta_num_1_cn = movePreprocessed(meta_df_1_cn, path, "15T", "CN", divider="Br_")